In [ ]:
# import os

# # Check the contents of the train and val directories
# train_dir = '/content/data/train'
# val_dir = '/content/data/val'

# print("Train directory classes:", os.listdir(train_dir))
# print("Validation directory classes:", os.listdir(val_dir))


In [ ]:
# !find /content/data -name '.ipynb_checkpoints' -type d -exec rm -rf {} +


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Dataset paths
train_dir = '/content/data/train'
val_dir = '/content/data/val'

# Image augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescale for validation data
val_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',  # Multi-class classification
    classes=['fire', 'smoke', 'default']
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    classes=['fire', 'smoke', 'default']
)

# Load the pre-trained MobileNetV2 model without the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the base model layers
base_model.trainable = False

# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax')(x)  # 3 output classes: fire, smoke, default

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Found 1048 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 128, 128, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 64, 64, 32)     │            864 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 64, 64, 32)     │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 64, 64, 32)     │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 64, 64, 32)     │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 64, 64, 32)     │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 64, 64, 32)     │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 64, 64, 16)     │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 64, 64, 16)     │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 64, 64, 96)     │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 64, 64, 96)     │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 64, 64, 96)     │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 65, 65, 96)     │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 32, 32, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├──────────────────────

 Total params: 2,422,339 (9.24 MB)

 Trainable params: 164,355 (642.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
base_model.trainable = True
# Freeze only the first few layers
for layer in base_model.layers[:100]:
    layer.trainable = False


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
history_finetune = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)


Epoch 1/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 32s 903ms/step - accuracy: 0.8340 - loss: 0.4381 - val_accuracy: 0.3743 - val_loss: 1.4250
Epoch 2/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 41s 890ms/step - accuracy: 0.8078 - loss: 0.5009 - val_accuracy: 0.4035 - val_loss: 1.4231
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 32s 915ms/step - accuracy: 0.8462 - loss: 0.4367 - val_accuracy: 0.3860 - val_loss: 1.4285
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 41s 914ms/step - accuracy: 0.8514 - loss: 0.4179 - val_accuracy: 0.4035 - val_loss: 1.4105
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 41s 929ms/step - accuracy: 0.8536 - loss: 0.4124 - val_accuracy: 0.4386 - val_loss: 1.3752
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 44s 989ms/step - accuracy: 0.8587 - loss: 0.3856 - val_accuracy: 0.4503 - val_loss: 1.3485
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.8375 - loss: 0.4071 - val_accuracy: 0.4678 - val_loss: 1.3511
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 34s 889ms/step - accuracy: 0.8666 - loss: 0.3535 - val_accurac

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

class_names = ['fire', 'smoke', 'default']

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions)]
    confidence = np.max(predictions) * 100

    print(f"Predicted Class: {predicted_class}")
    print(f"Confidence: {confidence:.2f}%")
    return predicted_class

# Test the model on a sample image
test_image_path = '/content/data/val/default/img_220.jpg'
predict_image(test_image_path)

test_image_path = '/content/data/val/fire/img_139.jpg'
predict_image(test_image_path)

test_image_path = '/content/data/val/smoke/img_264.jpg'
predict_image(test_image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted Class: default
Confidence: 98.06%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Predicted Class: fire
Confidence: 90.63%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted Class: smoke
Confidence: 96.12%


'smoke'